In [38]:
import sys
sys.path.append('/mnt/data/workspace/trip-overview/src/')
print(sys.path)

['/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/trip-overview/prototype', '/home/valentin/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.2031190714/pythonFiles/vscode_datascience_helpers', '/home/valentin/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.2031190714/pythonFiles', '/home/valentin/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.2031190714/pythonFiles/lib/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/mnt/data/workspace/trip-overview/.venv/lib/python3.8/site-packages', '/mnt/data/workspace/trip-overview/.venv/lib/python3.8/site-packages/IPython/extensions', '/home/valentin/.ipython', '/mnt/data/workspace/trip-overview/src/generate_site.py', '/mnt/data/workspace/trip-overview/src/methods.py', '/mnt/data/workspace/trip-overview/src/methods.py', '/mnt/data/workspace/trip-overview/src/methods.py', '/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/t

In [130]:
import pandas as pd
from math import sin, cos, sqrt, atan2, radians

def dist_from_gps(coord_a, coord_b):
    """
    Haversine’s algorithm
    param: decimal gps coord_a [lat, lon]
    param: decimal gps coord_b [lat, lon]
    result: distance between two gps coords
    """
    R = 6373.0 # approximate radius of earth in km

    lat1 = radians(coord_a[0])
    lon1 = radians(coord_a[1])
    lat2 = radians(coord_b[0])
    lon2 = radians(coord_b[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

gps_trace = pd.read_csv("gps.csv", header=None, names=["latitude", "longitude", "none"])[["latitude", "longitude"]]
gps_trace["altitude"] = 0.0
gps_trace["speed"] = 0.0
gps_trace["km"] = 0.0
for i in range(1, len(gps_trace["longitude"])):
    gps_trace["km"].iloc[i] = gps_trace["km"].iloc[i-1] + dist_from_gps(
        [gps_trace["latitude"].iloc[i-1], gps_trace["longitude"].iloc[i-1]],
        [gps_trace["latitude"].iloc[i], gps_trace["longitude"].iloc[i]])
gps_trace["current_country"] = "France"
gps_trace["current_step"] = 0
gps_trace['timestamp'] = range(1, 1 + len(gps_trace))
gps_trace

,latitude,longitude,altitude,speed,km,current_country,current_step,timestamp
0,48.87736,2.27966,0.0,0.0,0.000000,France,0,1
1,48.87736,2.27966,0.0,0.0,0.000000,France,0,2
2,48.87721,2.27951,0.0,0.0,0.019969,France,0,3
3,48.87720,2.27953,0.0,0.0,0.021807,France,0,4
4,48.87715,2.27967,0.0,0.0,0.033461,France,0,5
...,...,...,...,...,...,...,...,...
2720,48.54520,3.67221,0.0,0.0,129.176174,France,0,2721
2721,48.54501,3.67182,0.0,0.0,129.211831,France,0,2722
2722,48.54466,3.67063,0.0,0.0,129.307718,France,0,2723
2723,48.54455,3.67000,0.0,0.0,129.355697,France,0,2724


In [152]:
import folium
import folium.plugins
import base64
import branca


def get_road_trip_gps_trace(trace):
    # Change timestamp to datetime
    trace['date'] = pd.to_datetime(trace['timestamp'], unit='s')
    trace.reset_index(drop=True, inplace=True)
    # Interpolate the correct columns
    trace[["latitude", "longitude", "altitude", "speed", "km"]] = trace[["latitude", "longitude", "altitude", "speed", "km"]].interpolate(
        method='linear')
    # Fill forward current_country and current_step
    trace["current_country"].fillna(method='ffill', inplace=True)
    trace["current_step"].fillna(method='ffill', inplace=True)
    # Reset the current_step type
    trace["current_step"] = trace["current_step"].astype('int64')
    # Set current_step as index
    trace = trace.set_index(['current_step'])
    return trace

def get_sleeping_locations(data, static_position_threshold=1, min_distance=10):
    for current_step_trace_idx in range(gps_trace_clean.index.values[-1] + 1):
        current_step_trace = gps_trace_clean.loc[current_step_trace_idx]
    return 
    sleeping_df = data
    # Filter the static position
    sleeping_df = sleeping_df[sleeping_df.speed <= static_position_threshold]
    # Get the dates
    sleeping_df['date'] = sleeping_df["timestamp"].apply(lambda x: pd.to_datetime(x, unit="s").date())
    # Filter the last position of the day
    sleeping_df = sleeping_df.drop_duplicates(subset=["date"], keep='last')
    # Compute distance to each other
    sleeping_df["dist_from_last"] = 0.0
    for i in range(1, len(sleeping_df[["latitude", "longitude"]])):
        sleeping_df["dist_from_last"].iloc[i] = distance(sleeping_df[["latitude", "longitude"]].iloc[i - 1].values,
                                                            sleeping_df[["latitude", "longitude"]].iloc[i].values)
    # Filter positions that distance is sufficient
    sleeping_df = sleeping_df[sleeping_df.dist_from_last >= min_distance]
    return sleeping_df[["timestamp", "latitude", "longitude", "altitude"]].copy()

gps_trace_clean = get_road_trip_gps_trace(gps_trace)
sleeping = get_sleeping_locations(gps_trace)

center_of_map = gps_trace_clean[["latitude", "longitude"]].iloc[-1].tolist() # Last updated GPS position
whole_trip_trace = gps_trace_clean[["latitude", "longitude"]]
online_map = folium.Map(center_of_map, tiles="OpenStreetMap", attr="Capsule map")

# TODO Handle case where the sub step trace length is not > 1
for current_step_trace_idx in range(gps_trace_clean.index.values[-1] + 1):
    current_step_trace = gps_trace_clean.loc[current_step_trace_idx]
    if len(current_step_trace) > 1:
        distance_traveled_in_step = current_step_trace["km"].iloc[-1] - current_step_trace["km"].iloc[0]
        date = current_step_trace["date"].iloc[0].day
        """gif = "test.gif" # TODO
        tooltip_html = '<h1>{date}</h1><p>Etape {step}</p><p>Distance parcourue {distance} km</p><img src={gif}>'\
            .format(date=distance_traveled_in_step,
                    step=current_step_trace_idx,
                    distance=distance_traveled_in_step,
                    gif=gif)"""
        folium.plugins.AntPath(
            locations=current_step_trace[["latitude", "longitude"]],
            dash_array=[10, 15],
            delay=800,
            weight=6,
            color="#F6FFF3",
            pulse_color="#000000",
            paused=False,
            reverse=False,
            #tooltip=tooltip_html TODO
        ).add_to(online_map)

folium.LayerControl(collapsed=False).add_to(online_map)

folium.plugins.LocateControl().add_to(online_map)

folium.plugins.Fullscreen(
    title="Agrandir",
    title_cancel="Annuler",
    force_separate_button=True,
).add_to(online_map)

# Limit bounds
online_map.fit_bounds(online_map.get_bounds())
online_map

In [ ]:
m3 = folium.Map(arr[1], zoom_start=10)

folium.plugins.AntPath(
    locations=arr, 
    dash_array=[10, 15],
    delay=800,
    weight=6,
    color="#F6FFF3",
    pulse_color="#000000",
    paused=False,
    reverse=False,
    tooltip='<h1>10 Décembre 2021</h1><p>Etape X</p><p>Distance parcourue X km</p><img src="road.gif">'
).add_to(m3)
m3.fit_bounds(m3.get_bounds())


marker_cluster = MarkerCluster(name='Photos during the trip').add_to(m3)
marker_cluster_steps = FeatureGroup(name="Trip steps").add_to(m3)
marker_unesco_cluster = MarkerCluster(name='Unesco sites').add_to(m3)

encoded = base64.b64encode(open("moon.jpg", 'rb').read())
html = '<img src="data:image/jpeg;base64,{}">'.format
resolution, width, height = 15, 50, 25
iframe = IFrame(html(encoded.decode('UTF-8')), width=(width*resolution)+20, height=(height*resolution)+20)
popup = folium.Popup(iframe, max_height=1000)

kw = {"prefix": "fa", "color": "green", "icon": "camera"}
# Create photos marker
for i in range(0, len(arr), 200):
    loc = list(arr[i])
    loc[0] += random.uniform(-0.5, 0.5)
    loc[1] += random.uniform(-0.5, 0.5)
    
    encoded = base64.b64encode(open("moon.jpg", 'rb').read())
    html = '<h1>10 Décembre 2021</h1><p>Etape X</p><p>Distance parcourue X km</p><p>Coordonnée GPS: {}, {}</p><p><img src="data:image/jpeg;base64,{}"></p>'.format
    icon = folium.Icon(**kw)
    folium.Marker(location=loc, icon=icon, tooltip=html(loc[0], loc[1], encoded.decode('UTF-8'))).add_to(marker_cluster)

kw = {"prefix": "fa", "color": "blue", "icon": "bed"}
# Create sleep steps
for i in range(0, len(arr), 2000):
    loc = list(arr[i])
    html = '<h1>10 Décembre 2021</h1><p>Etape X</p><p>Distance parcourue X km</p><p>Coordonnée GPS: {}, {}</p>'.format(loc[0], loc[1])
    icon = folium.Icon(**kw)
    folium.Marker(location=loc, icon=icon, tooltip=html).add_to(marker_cluster_steps)

kw = {"prefix": "fa", "color": "red", "icon": "university"}

for i in range(0, len(unesco_sites)):
    loc = [unesco_sites["latitude"][i], unesco_sites["longitude"][i]]
    icon = folium.Icon(**kw)
    folium.Marker(location=loc, icon=icon, popup=folium.Popup(unesco_sites["short_description_fr"][i], max_width=1000), tooltip=unesco_sites["name_fr"][i]+" \n click for more details").add_to(marker_unesco_cluster)
    
    
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 10px;
    left: 10px;
    width: 250px;
    height: 100px;
    z-index:9999;
    font-size:14px;
    ">
    <p>Temps de voyage: 54 jours</p>
    <p>Kilomètre parcourue: 13141 km</p>
    <p>Pays traversés: 5</p>
    <p>Date de mise à jour: 10 mars 2021</p>
</div>
<div style="
    position: fixed;
    bottom: 0px;
    left: 0px;
    width: 250px;
    height: 120px;
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    opacity: 0.7;
    ">
</div>
{% endmacro %}
"""

legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)
m3.get_root().add_child(legend)


folium.LayerControl(collapsed=False).add_to(m3)

plugins.LocateControl().add_to(m3)

plugins.Fullscreen(
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m3)

m3.save("test.html")

m3